In [46]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [179]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [180]:
data = pd.read_csv("/content/drive/MyDrive/Intrade/Reliance/reliance_1day.csv")
holidays = pd.read_csv("/content/drive/MyDrive/Intrade/Off_days/holidays.csv")
trade_on_holiday = pd.read_csv("/content/drive/MyDrive/Intrade/Off_days/trade_on_holidays.csv")
trade_on_weekend = pd.read_csv("/content/drive/MyDrive/Intrade/Off_days/trade_on_weekends.csv")
missing_dates = pd.read_csv("/content/drive/MyDrive/Intrade/Off_days/missing_dates.csv")

In [181]:
data

,datetime,stock_code,open,high,low,close,volume
0,2007-01-02 12:01:00,RELIND,318.47,321.00,317.37,319.85,1149902
1,2007-01-03 12:01:00,RELIND,320.81,322.20,318.55,320.71,1489730
2,2007-01-04 12:01:00,RELIND,321.98,324.33,318.94,319.76,3022423
3,2007-01-05 12:01:00,RELIND,318.47,327.14,318.47,321.55,3047126
4,2007-01-08 12:01:00,RELIND,322.92,322.92,316.50,318.90,2191355
...,...,...,...,...,...,...,...
3851,2022-11-01 12:11:00,RELIND,2600.00,2600.00,2508.00,2529.85,6443999
3852,2022-11-02 12:11:00,RELIND,2532.30,2550.20,2515.00,2545.65,4145144
3853,2022-11-03 12:11:00,RELIND,2529.85,2559.55,2525.05,2555.15,3089026
3854,2022-11-04 12:11:00,RELIND,2553.90,2600.00,2551.10,2592.75,5803737


In [182]:
data['datetime'] =  pd.to_datetime(data['datetime'], infer_datetime_format=True)
cols = ['datetime','open','high','low','close']
data = data[cols]
data = data.set_index("datetime")

holidays['date'] = holidays['date'].astype('datetime64[ns]')

In [183]:
data

,open,high,low,close
datetime,,,,
2007-01-02 12:01:00,318.47,321.00,317.37,319.85
2007-01-03 12:01:00,320.81,322.20,318.55,320.71
2007-01-04 12:01:00,321.98,324.33,318.94,319.76
2007-01-05 12:01:00,318.47,327.14,318.47,321.55
2007-01-08 12:01:00,322.92,322.92,316.50,318.90
...,...,...,...,...
2022-11-01 12:11:00,2600.00,2600.00,2508.00,2529.85
2022-11-02 12:11:00,2532.30,2550.20,2515.00,2545.65
2022-11-03 12:11:00,2529.85,2559.55,2525.05,2555.15


In [184]:
t = data.groupby(pd.Grouper(freq='1D')).agg({"open": "first", 
                                             "high": "max", 
                                             "low": "min", 
                                             "close": "last"})
t

,open,high,low,close
datetime,,,,
2007-01-02,318.47,321.00,317.37,319.85
2007-01-03,320.81,322.20,318.55,320.71
2007-01-04,321.98,324.33,318.94,319.76
2007-01-05,318.47,327.14,318.47,321.55
2007-01-06,NaN,NaN,NaN,NaN
...,...,...,...,...
2022-11-03,2529.85,2559.55,2525.05,2555.15
2022-11-04,2553.90,2600.00,2551.10,2592.75
2022-11-05,NaN,NaN,NaN,NaN


In [185]:
"""t_before = t[t.index.date < datetime.date(2022,10,21)]
t_after = t[t.index.date > datetime.date(2011,1,3)]

t_before = t_before[t_before.index.time < datetime.time(15,30)]
t_before = t_before[t_before.index.time > datetime.time(9,14)]

t_after = t_after[t_after.index.time < datetime.time(15,30)]
t_after = t_after[t_after.index.time > datetime.time(9,14)]"""

't_before = t[t.index.date < datetime.date(2022,10,21)]\nt_after = t[t.index.date > datetime.date(2011,1,3)]\n\nt_before = t_before[t_before.index.time < datetime.time(15,30)]\nt_before = t_before[t_before.index.time > datetime.time(9,14)]\n\nt_after = t_after[t_after.index.time < datetime.time(15,30)]\nt_after = t_after[t_after.index.time > datetime.time(9,14)]'

In [186]:
"""combined_1day = pd.concat([t_before, t_after])
combined_1day = combined_1day.sort_index(ascending=True)
combined_1day.isnull().sum()"""

'combined_1day = pd.concat([t_before, t_after])\ncombined_1day = combined_1day.sort_index(ascending=True)\ncombined_1day.isnull().sum()'

In [187]:
combined_1day = t

In [188]:
combined_1day_weekdays = combined_1day[combined_1day.index.dayofweek < 5]
combined_1day_weekends = combined_1day[combined_1day.index.dayofweek > 4]

combined_1day_weekends = combined_1day_weekends[combined_1day_weekends['open'].notna()]
combined_1day = pd.concat([combined_1day_weekdays,combined_1day_weekends])
combined_1day = combined_1day.sort_index(ascending=True)

In [189]:
combined_1day.isnull().sum()

open     293
high     293
low      293
close    293
dtype: int64

In [190]:
holidays_list = pd.to_datetime(holidays['date']).dt.date.unique().tolist()
combined_1day_holidays = combined_1day[combined_1day.index.floor('D').isin(holidays_list)]

combined_1day_holidays = combined_1day_holidays[combined_1day_holidays['open'].notna()]

combined_1day_nonholidays = combined_1day[~combined_1day.index.floor('D').isin(holidays_list)]
combined_1day = pd.concat([combined_1day_nonholidays,combined_1day_holidays])
combined_1day = combined_1day.sort_index(ascending=True)

In [192]:
missing_dates_list = pd.to_datetime(missing_dates['reliance']).dt.date.unique().tolist()
combined_1day_missing_dates = combined_1day[combined_1day.index.floor('D').isin(missing_dates_list)]

combined_1day_missing_dates = combined_1day_missing_dates[combined_1day_missing_dates['open'].notna()]

combined_1day_nonmissing_dates = combined_1day[~combined_1day.index.floor('D').isin(missing_dates_list)]
combined_1day = pd.concat([combined_1day_nonmissing_dates,combined_1day_missing_dates])
combined_1day = combined_1day.sort_index(ascending=True)

In [194]:
#trade on holiday
combined_1day_trade_on_holiday = t.loc[trade_on_holiday['Date']]
combined_1day_trade_on_holiday
combined_1day = pd.concat([combined_1day,combined_1day_trade_on_holiday])
combined_1day = combined_1day.sort_index(ascending=True)

In [195]:
t.loc[trade_on_weekend['Date']]
#trade on weekend
combined_1day_trade_on_weekend = t.loc[trade_on_weekend['Date']]
combined_1day_trade_on_weekend
combined_1day = pd.concat([combined_1day,combined_1day_trade_on_weekend])
combined_1day = combined_1day.sort_index(ascending=True)

In [196]:
combined_1day

,open,high,low,close
datetime,,,,
2007-01-02,318.47,321.00,317.37,319.85
2007-01-03,320.81,322.20,318.55,320.71
2007-01-04,321.98,324.33,318.94,319.76
2007-01-05,318.47,327.14,318.47,321.55
2007-01-08,322.92,322.92,316.50,318.90
...,...,...,...,...
2022-11-01,2600.00,2600.00,2508.00,2529.85
2022-11-02,2532.30,2550.20,2515.00,2545.65
2022-11-03,2529.85,2559.55,2525.05,2555.15


In [61]:
'''df = df[df.isnull().any(axis=1)] #Check for null rows

#saving csv files
df.to_csv('null_reliance.csv', encoding='utf-8')
df.to_csv('null_nifty.csv', encoding='utf-8')

#reading csv files
reliance = pd.read_csv('null_reliance.csv') 
nifty = pd.read_csv('null_nifty.csv')

#rename column of datetime to stock name
reliance.rename(columns = {'datetime':'reliance'}, inplace = True)
nifty.rename(columns = {'datetime':'reliniftyance'}, inplace = True)

#taking 1st columns
reliance = reliance['reliance']
nifty = nifty['nifty']

#concat both dataframe
df1 = pd.concat([reliance, nifty], axis=1)

#save
df1.to_csv('missing_dates.csv', encoding='utf-8', index=False)'''